# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create is assigned to jmfansi@gmail.com and will expire on April 03, 2017. For commercial licensing options, visit https://dato.com/buy/.


[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1465652986.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.630792     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.709357     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [28]:
all_coefficients = model_all.get('coefficients')
non_ero_weight_features = all_coefficients[all_coefficients['value']!=0.0]['name']
print 'Non_Zero_Weight (%d)' % non_ero_weight_features.size()
print non_ero_weight_features
zer_weight_features = all_coefficients[all_coefficients['value']==0.0]['name']
print 'Zero_Weight (%d)' % zer_weight_features.size()
print zer_weight_features

Non_Zero_Weight (6)
['(intercept)', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above']
Zero_Weight (12)
['bedrooms', 'bedrooms_square', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'sqft_basement', 'yr_built', 'yr_renovated']


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

ANS:'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above'

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [29]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [32]:
import numpy as np
min_errors = None
best_l1 = None
for l1_penalty in np.logspace(1, 7, num=13):
    #Fitting the model
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty)
    #computing the RSS
    predictions = model.predict(validation)
    errors = validation['price']-predictions
    RSS = np.dot(errors, errors)
    if(min_errors == None or RSS < min_errors):
        min_errors = RSS
        best_l1 = l1_penalty
print 'The Best L1 Penalty is %f. This produces an RSS of %f' % (best_l1, min_errors)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.542946     | 5659149.988446     | 285408.885485 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

The Best L1 Penalty is 10.000000. This produces an RSS of 625766285142460.500000


In [33]:
best_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10.0)
#computing the RSS
predictions = best_model.predict(testing)
errors = testing['price']-predictions
RSS = np.dot(errors, errors)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.464557     | 5737572.318059     | 289374.784200 |

| 6         | 7        | 0.000003  | 0.497439     | 5655864.782013     | 285354.826809 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?

ANS:10.000000
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

ANS: 1.56983602382e+14


In [34]:
print RSS

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

ANS: 18

In [38]:
best_model_coeff = best_model.get('coefficients')
non_ero_weight_features_2 = best_model_coeff[best_model_coeff['value']!=0.0]['name']
print 'there are %d non zeros features' % (non_ero_weight_features_2.size())
print non_ero_weight_features_2

there are 18 non zeros features
['(intercept)', 'bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [36]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [65]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [66]:
non_zeroes = []
for l1_penalty in np.logspace(8, 10, num=20):
    #Fitting the model
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty)
    non_zeroes.append(model['coefficients']['value'].nnz())

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000012  | 0.339155     | 7239807.218168     | 388380.843122 |

| 6         | 7        | 0.000012  | 0.359607     | 7187431.218291     | 380705.223778 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [70]:
l1_penalty =  np.logspace(8, 10, num=20)
largest_non_zero_index=0
largest_l1_penalty_with_largest_non_zero=None
smallest_l1_penalty_with_smallest_non_zero=None
index=0
for nz in non_zeroes:   
    if nz > max_nonzeros: 
        current_max_l1 = l1_penalty[index]
        if(largest_l1_penalty_with_largest_non_zero == None or current_max_l1 > largest_l1_penalty_with_largest_non_zero):
            largest_l1_penalty_with_largest_non_zero = current_max_l1
    elif nz < max_nonzeros:
        current_min_l1 = l1_penalty[index]
        if(smallest_l1_penalty_with_smallest_non_zero == None or current_min_l1 < smallest_l1_penalty_with_smallest_non_zero):
            smallest_l1_penalty_with_smallest_non_zero=current_min_l1
    index = index +1

l1_penalty_min = largest_l1_penalty_with_largest_non_zero
l1_penalty_max = smallest_l1_penalty_with_smallest_non_zero
print 'l1 penalty min is %f' % (l1_penalty_min)
print 'l1 penalty max is %f' % (l1_penalty_max)


l1 penalty min is 2976351441.631313
l1 penalty max is 3792690190.732254


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

ANS: (l1_penalty_min, l1_penalty_max) = (2976351441.631313, 3792690190.732254)

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [71]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [76]:
min_rss = None
select_l1= None
selectedModel = None
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                             l2_penalty=0., l1_penalty=l1_penalty)
    
    if (model['coefficients']['value'].nnz()-1 == max_nonzeros):
        #computing the RSS
        predictions = model.predict(validation)
        errors = validation['price']-predictions
        RSS = np.dot(errors, errors)
        if(min_rss==None or RSS < min_rss):
            min_rss=RSS
            select_l1 = l1_penalty
            selectedModel = model

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.420567     | 6700050.222874     | 372113.713871 |

| 6         | 7        | 0.000003  | 0.449598     | 6697293.370032     | 370685.051067 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [82]:
print min_rss
print select_l1
all_coef = selectedModel['coefficients']
non_zero = all_coef[all_coef['value']!=0.0]['name']
print non_zero
print non_zero.size()

1.03461690923e+15
3320073020.2
['(intercept)', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'floors', 'grade', 'sqft_above', ... ]
8
